In [258]:
import pandas as pd
import numpy as np

In [259]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
#Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";") -> zu viele Datenpunkte fehlen
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/1000006cleaned.csv")

In [260]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()




,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [261]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,-7.0,-6.0,-15.0,-19.0,0.0,-2.0,-2.0,-3.0,-1.0,-23.0,-5.0,-10.0,-4.0,-7.0,0.0,-2.0
1,2022-01-01 01:00:00+00:00,-7.0,-1.0,-4.0,-9.0,-1.0,0.0,0.0,0.0,0.0,0.0,-9.0,-19.0,0.0,-8.0,-1.0,-1.0
2,2022-01-01 02:00:00+00:00,-4.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-12.0,-23.0,0.0,-8.0,0.0,0.0
3,2022-01-01 03:00:00+00:00,-3.0,0.0,-1.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,-10.0,0.0,-2.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,-2.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-7.0,0.0,-2.0,0.0,1.0


In [262]:
Flugbewegungen.head()
Verkehrsdaten.head()
print(type(Verkehrsdaten))

<class 'pandas.core.frame.DataFrame'>


In [263]:
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())


0


In [264]:
#standardise the data
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten = dfs

# Save the scalers dictionary for later use
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [265]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [266]:
Verkehrsdaten.head()


,Datum,350n_sumPW,350n_sumLief,350n_sumLW,350v_sumPW,350v_sumLief,350v_sumLW,352n_sumPW,352n_sumLief,352n_sumLW,...,84111104n_sumLW,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW
0,2021-12-31 23:00:00+00:00,-0.429021,-0.940608,-0.796961,-0.888120,-0.953473,-0.810167,0.046689,-0.602995,-0.450842,...,-0.085348,-1.36395,1.323365,-0.331206,-0.892472,-0.571421,-1.055084,-1.120734,-0.654501,-0.611608
1,2022-01-01 00:00:00+00:00,-0.548987,-0.970116,-0.631220,-0.731198,-0.774723,-0.732415,-0.145855,-0.648232,-1.020804,...,-0.085348,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471
2,2022-01-01 01:00:00+00:00,-0.867158,-0.911100,-0.796961,-0.935672,-0.794584,-0.810167,-0.338398,-0.783942,-1.020804,...,-0.284046,-1.36395,0.745474,-0.744920,-0.955306,-0.781539,-1.055084,-1.048762,-0.773270,-1.072471
3,2022-01-01 02:00:00+00:00,-1.117523,-0.970116,-0.796961,-1.049798,-1.032917,-0.732415,-0.675350,-0.693469,-1.115797,...,-0.681442,-1.36395,0.575505,-0.744920,-0.937353,-0.571421,-1.055084,-1.057758,-0.416962,-1.072471
4,2022-01-01 03:00:00+00:00,-1.185330,-0.881591,-0.796961,-1.344621,-1.052778,-0.810167,-0.883939,-0.874416,-1.115797,...,-0.482744,-1.36395,0.269563,-0.744920,-1.045068,-0.571421,-1.055084,-1.201702,-0.416962,-1.072471


In [ ]:
Parkhäuser['Datum'] = pd.to_datetime(Parkhäuser['Datum'], errors='coerce')
Wetterdaten['Datum'] = pd.to_datetime(Wetterdaten['Datum'], errors='coerce')
Flugbewegungen['Datum'] = pd.to_datetime(Flugbewegungen['Datum'], errors='coerce')
Gasverbrauch['Datum'] = pd.to_datetime(Gasverbrauch['Datum'], errors='coerce')
Stromverbrauch['Datum'] = pd.to_datetime(Stromverbrauch['Datum'], errors='coerce')
Verkehrsdaten['Datum'] = pd.to_datetime(Verkehrsdaten['Datum'], errors='coerce')



# Convert timezone but keep time by explicitly formatting the datetime
Parkhäuser['Datum'] = Parkhäuser['Datum'].dt.tz_convert(None)
Wetterdaten['Datum'] = Wetterdaten['Datum'].dt.tz_convert(None)
Flugbewegungen['Datum'] = Flugbewegungen['Datum'].dt.tz_convert(None)
Gasverbrauch['Datum'] = Gasverbrauch['Datum'].dt.tz_convert(None)
Stromverbrauch['Datum'] = Stromverbrauch['Datum'].dt.tz_convert(None)
Verkehrsdaten['Datum'] = Verkehrsdaten['Datum'].dt.tz_convert(None)



# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Flugbewegungen, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
# Display the first few rows of the merged dataframe
final_df.head(1)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Stunde
0,2022-01-01,-0.48985,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.15826,-0.151263,...,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,0


In [272]:
final_df["Hour"].head(25)

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24     0
Name: Hour, dtype: int64

In [273]:

final_df['Datum'] = final_df.apply(lambda row: row['Datum'].replace(hour=row['Hour']), axis=1)

# Drop the Hour column if no longer needed
final_df = final_df.drop('Hour', axis=1)

# Display the updated DataFrame
print(final_df.head())

                Datum  Rebgassechange  Clarahuuschange  Citychange  \
0 2022-01-01 00:00:00        -0.48985        -1.162123   -0.163777   
1 2022-01-01 01:00:00        -0.48985        -1.162123   -0.163777   
2 2022-01-01 02:00:00        -0.48985        -1.162123   -0.163777   
3 2022-01-01 03:00:00        -0.48985        -1.162123   -0.163777   
4 2022-01-01 04:00:00        -0.48985        -1.162123   -0.163777   

   Storchenchange  Post Baselchange  Aeschenchange  Bahnhof Südchange  \
0       -1.110018         -0.000078      -0.258065          -0.245463   
1       -1.110018         -0.000078      -0.258065          -0.245463   
2       -1.110018         -0.000078      -0.258065          -0.245463   
3       -1.110018         -0.000078      -0.258065          -0.245463   
4       -1.110018         -0.000078      -0.258065          -0.245463   

   Bad. Bahnhofchange  Europechange  ...  84111104v_sumLief  84111104v_sumLW  \
0            -0.15826     -0.151263  ...           1.425346 

# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

In [276]:
final_df.to_csv("../Working_DataFrame.csv", index=False)

In [270]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen
